# Assignment 2: Feedforward Neural Networks

## Programming Question 1: Implementing Deep FNN
Through this assignment you learn how to use `torch.nn` module of torch to implement a neural network in only few lines of code.

We first import all required packages.

In [63]:
# We import torch and the nn module
import torch
import torch.nn as nn

# We import NumPy and tqdm
import numpy as np
from tqdm import tqdm

# We need few items from Scikit-Learn
import sklearn.datasets as DataSets
from sklearn.model_selection import train_test_split

# Also we need to plot few curves
import matplotlib.pyplot as plt

### Data Splitting: Make Training Mini-Batches

In [64]:
dataset = DataSets.make_classification(n_samples = 10000
                                       , n_features = 20
                                       , n_informative=5
                                       , n_redundant=15
                                       , random_state=1)
X, v = dataset
print(X.size, v.size, X.dtype, v.dtype, X[0,0], v[0])
v = v.astype(np.float64)
print(X.dtype, v.dtype)


200000 10000 float64 int64 2.712949015269261 1
float64 float64


Below you can implement the function

In [65]:
def data_splitter(X, v, batch_size, train_size):
    '''
    X is list of data-points
    v is list of labels
    train_size is the fraction of data-points used for training
    '''
    X_train, X_test, v_train, v_test = train_test_split(X, v
        , train_size= 0.8
        , shuffle = True
        )
    X_train = torch.tensor(X_train,dtype=torch.float32)
    v_train = torch.tensor(v_train,dtype=torch.float32).reshape(-1,1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    v_test = torch.tensor(v_test, dtype=torch.float32).reshape(-1,1)
    batch_indx = torch.arange(X_train.shape[0])
    return X_train, v_train, X_test, v_test, batch_indx

### Basic Classes

For this task you can use `nn.Layer()`. You mainly need to complete the following `class`.

In [66]:
class myClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Linear(20,64)
        self.active1 = nn.ReLU()

        self.layer2 = nn.Linear(64,32)
        self.active2 = nn.ReLU()

        self.output = nn.Linear(32,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.active1(self.layer1(x))
        x = self.active2(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

### Backpropagation
We now learn how we can use the autograd of PyTorch to implement backpropagation. Simply complete the following code and run it to see how autograd works.

In [67]:
# first, instantiate the model 
myModel = myClassifier()

# now, define the loss
loss_fn = nn.BCELoss()

# split the dataset via function data_splitter()
X_train, v_train, X_test, v_test, batch_indx = data_splitter(X, v, 100, 0.8)

# take a sample point from training dataset
x = X_train[0].unsqueeze(0)   
v = v_train[0].unsqueeze(0)  

# forward pass
y = myModel(x)
Loss = loss_fn(y, v)

# backward pass
Loss.backward()


We can now access the computed gradient of the loss with respect to the output bias using the following code.

In [68]:
myModel.output.bias.grad

tensor([-0.4847])

### Optimizer
We can further define the optimizer for our model as follows:

In [69]:
# first, instantiate the model 
myModel = myClassifier()

# now define the optimizer
optimizer = torch.optim.Adam(myModel.parameters()
                        , lr=0.0001 # this specifies the learning rate
                        )

### Implementing Training and Test Loop
Now that we have all the components, we can simply implement the training loop. Complete the following code to get the training done.

In [ ]:
def training_loop(model):
    # define the loss and optimizer
    loss_fn = nn.BCELoss() # binary cross-entropy
    optimizer = # complete <use Adam>

    # set the training parameters
    n_epochs = 300   # number of epochs
    batch_size = 40  # batch size

    # specify training and test datasets and the batch indices
    # use data_splitter() and X, v are generated by Scikit-Learn
    X_train, v_train, X_test, v_test, batch_indx = # complete 

    # make empty list to save training and test risk
    train_risk = []
    test_risk = []

    # training loop

    # we visualize the training progress via tqdm
    with tqdm(range(n_epochs), unit="epoch") as epoch_bar:
        epoch_bar.set_description("training loop")
        for epoch in epoch_bar:

            # tell pytorch that you start training
            model.train()

            for indx in batch_indx:
                # take a batch of samples
                X_batch = # complete
                v_batch = # complete

                # pass forward the mini-batch
                y_batch = # complete

                # compute the loss
                loss = # complete

                # backward pass
                # first make gradient zero
                optimizer.zero_grad()
                # then, compute the gradient of loss
                # complete

                # now update weights by one optimization step
                optimizer.step()

            # we are done with one epoch
            # we now evaluate training and test risks
            # first we tell pytorch we are doing evaluation
            model.eval()

            # now we evaluate the training risk
            y_train = # complete
            CE_train = # complete
            train_risk.append(CE_train.item())

            # then we evaluate the test risk
            y_test = # complete
            CE_test = # complete
            test_risk.append(CE_test.item())
        return train_risk, test_risk

We can now try training our model by passing the model to the training loop. Complete the following code to train the model and plot the learning curves.

In [ ]:
myModel = myClassifier()
train_risk, test_risk = # complete

# complete <plot training risk>
# complete <plot test risk>
plt.show()

## Programming Question 2: MNIST Dataset
We play around a bit with the datasets in PyTorch.

In [ ]:
import torch
import torchvision.datasets as DS
import torchvision.transforms as transform

Import the MNIST dataset by running the code below. 

In [ ]:
mnist = DS.MNIST('./data',train=True,transform=transform.ToTensor(),download=True)

We now build a function `myBatcher()` which gets the batch size as the input and returns a list containing multiple batches. 

In [ ]:
def myBatcher(batch_size):
    batch_list = []
    num_batches = # COMPLETE
    
    for j in range(num_batches):
        batch_x = torch.zeros(batch_size,784)
        batch_v = torch.zeros(batch_size)
        for i in range(batch_size):
            batch_x[i] = # COMPLETE
            batch_v[i] = # COMPLETE
            
        batch = (batch_x,batch_v)
        batch_list.append(batch)
    return batch_list

Let's try the implemented function

In [ ]:
batch_list = myBatcher(100)